In [13]:
# selenium으로 키를 조작하기 위한 import
import selenium
print(selenium.__version__)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ETC
import time # to wait not to seem as "bot"
import csv

4.22.0


# 3. 역대 대통령 데이터
https://www.pa.go.kr/research/contents/speech/index.jsp

### 대통령별 체크박스 Xpath

In [14]:
# 전체 : //*[@id="prstNm0"]
# 이승만 : //*[@id="prstNm1"]
# 윤보선 : //*[@id="prstNm2"]
# 박정희 : //*[@id="prstNm3"]
# 최규하 : //*[@id="prstNm4"]
# 전두환 : //*[@id="prstNm5"]
# 노태우 : //*[@id="prstNm6"]
# 김영삼 : //*[@id="prstNm7"]
# 김대중 : //*[@id="prstNm8"]
# 노무현 : //*[@id="prstNm9"]
# 이명박 : //*[@id="prstNm10"]
# 박근혜 : //*[@id="prstNm11"]
# 문재인 : //*[@id="prstNm12"]

In [15]:
# search speech data of specific president
def search(president):
    global driver
    name_d = {"전체":0, "이승만":1, "윤보선":2, "박정희":3, "최규하":4, "전두환":5, 
              "노태우":6, "김영삼":7, "김대중":8, "노무현":9, "이명박":10, "박근혜":11, 
              "문재인":12}
    presid_num = name_d[f"{president}"]

    checkbox = driver.find_element(By.XPATH, f'//*[@id="prstNm{presid_num}"]')
    search = driver.find_element(By.XPATH, '//*[@id="frm"]/fieldset[1]/div[2]/div[5]/button[2]')
    checkbox.click()
    time.sleep(1)
    search.click()
    time.sleep(1)
    doc_button = driver.find_element(By.XPATH, '//*[@id="m-items2"]/li[2]/a')
    doc_button.click()

In [16]:
# Helper function to click page button
def click_section_button():
    global driver
    try:
        page_button = WebDriverWait(driver, 10).until( # Waiting 10sec maximum
            EC.element_to_be_clickable((By.XPATH, f'//*[@id="frm"]/nav[2]/ul/li[13]/a')))
        driver.execute_script("arguments[0].scrollIntoView();", page_button) # scroll
        time.sleep(1)
        driver.execute_script("arguments[0].click();", page_button)  # Click with JavaScript
    except TimeoutException:
        print(f"Section button not found")
        raise  # if TimeoutException : throw Exception & quit

In [23]:
# Helper function to click page button
def click_page_button(page_num):
    global driver
    try:
        if page_num == 1:
            page_button = WebDriverWait(driver, 10).until( # Waiting 10sec maximum
                EC.element_to_be_clickable((By.XPATH, f'//*[@id="frm"]/nav[2]/ul/li[{page_num+2}]/strong')))
        else:
            page_button = WebDriverWait(driver, 10).until( # Waiting 10sec maximum
                EC.element_to_be_clickable((By.XPATH, f'//*[@id="frm"]/nav[2]/ul/li[{page_num+2}]/a')))
        
        driver.execute_script("arguments[0].scrollIntoView();", page_button) # scroll
        time.sleep(1)
        driver.execute_script("arguments[0].click();", page_button)  # Click with JavaScript
    except TimeoutException:
        print(f"Page button {page_num} not found")
        raise  # if TimeoutException : throw Exception & quit

In [19]:
#1 첫페이지

driver = webdriver.Chrome()
driver.get('https://www.pa.go.kr/research/contents/speech/index.jsp')
time.sleep(2)

speech_data = []
idx = 1
president = "문재인"
#str(input("President's name you want : "))
search(president)

for title_num in range(1, 21):
    try:
        clik = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, f'//*[@id="M_More"]/tr[{title_num}]/td[5]/a')))
        driver.execute_script("arguments[0].scrollIntoView();", clik) # scroll
        time.sleep(1)
        driver.execute_script("arguments[0].click();", clik)  # Click with JavaScript

        o_title = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[1]/td')
        o_type = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[3]/td[1]')
        o_date = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[2]/td[1]')
        o_speech = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[4]/td')

        title = o_title.text
        type = o_type.text
        date = o_date.text
        speech = o_speech.text
        speaker = president

        d = {"index": idx, "title": title, "type": type, "date": date, "speaker": president, "speech": speech}
        speech_data.append(d)
        print(f"{idx}. {title}")
        idx += 1

        driver.back()
    except Exception as e:
        print(f"ERROR OCCURED {str(e)}")

1. 제19대 대통령 취임식
2. 비정규직 근로자와의 대화
3. 미세먼지 바로알기 교실 현장 방문
4. 국방부 순시
5. 합동참모본부 순시
6. 제37주년 5·18민주화운동 기념식
7. 노무현 대통령 서거 8주기 추도식
8. 수석·보좌관회의
9. 수석·보좌관회의
10. 제22회 바다의 날 기념식
11. 수석·보좌관회의
12. OECD 국제교통포럼(ITF) 사무총장 당선 축전
13. 치매국가책임제 SNS 메시지
14. 치매국가책임제 관련 현장 방문
15. 제62회 현충일 추념식
16. 용산소방서 방문 후 SNS 메시지
17. 현장 소방대원들과의 대화
18. 국가안전보장회의(NSC)
19. 제30주년 6·10민주항쟁 기념식
20. 2017년도 추가경정예산안 관련 국회 시정 연설


In [24]:
# 10개 페이지들

driver = webdriver.Chrome()
driver.get('https://www.pa.go.kr/research/contents/speech/index.jsp')
time.sleep(2)

speech_data = []
idx = 1
president = "문재인"
#str(input("President's name you want : "))
search(president)

for page_num in range(1, 11):
    click_page_button(page_num)
    try:
        for title_num in range(1, 21):
            try:
                clik = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f'//*[@id="M_More"]/tr[{title_num}]/td[5]/a')))
                driver.execute_script("arguments[0].scrollIntoView();", clik) # scroll
                time.sleep(1)
                driver.execute_script("arguments[0].click();", clik)  # Click with JavaScript

                o_title = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[1]/td')
                o_type = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[3]/td[1]')
                o_date = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[2]/td[1]')
                o_speech = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[4]/td')

                title = o_title.text
                type = o_type.text
                date = o_date.text
                speech = o_speech.text
                speaker = president

                d = {"index": idx, "title": title, "type": type, "date": date, "speaker": president, "speech": speech}
                speech_data.append(d)
                print(f"{idx}. {title}")
                idx += 1

                driver.back()
            except Exception as e:
                print(f"ERROR OCCURED {str(e)}")
    except Exception as e:
        print(f"ERROR OCCURED {str(e)}")

1. 제19대 대통령 취임식
2. 비정규직 근로자와의 대화
3. 미세먼지 바로알기 교실 현장 방문
4. 국방부 순시
5. 합동참모본부 순시
6. 제37주년 5·18민주화운동 기념식
7. 노무현 대통령 서거 8주기 추도식
8. 수석·보좌관회의
9. 수석·보좌관회의
10. 제22회 바다의 날 기념식
11. 수석·보좌관회의
12. OECD 국제교통포럼(ITF) 사무총장 당선 축전
13. 치매국가책임제 SNS 메시지
14. 치매국가책임제 관련 현장 방문
15. 제62회 현충일 추념식
16. 용산소방서 방문 후 SNS 메시지
17. 현장 소방대원들과의 대화
18. 국가안전보장회의(NSC)
19. 제30주년 6·10민주항쟁 기념식
20. 2017년도 추가경정예산안 관련 국회 시정 연설
21. 한미연합사령부 순시
22. 제19회 미주한인체전 축전
23. 제1회 전국 시·도지사 간담회
24. 6·15남북공동선언 17주년 기념식
25. 수석·보좌관회의
26. 아시아인프라투자은행(AIIB) 연차총회 개회식
27. 원자력발전소 고리1호기 영구 정지 선포식
28. 일자리위원회 위원 위촉장 수여식 및 제1차 회의
29. 6·25 한국전쟁 제67주년 국군 및 유엔군 참전유공자 위로연
30. 2017 무주 세계태권도선수권대회 개막식
31. 제28회 국무회의
32. (미국 방문) 장진호 전투 기념비 헌화
33. (미국 방문) 한미 비즈니스 서밋
34. 2017 글로벌 바이오 콘퍼런스 영상 축사
35. 2017 사회적경제 주간 기념 사회적경제 박람회 개막식 영상 축사
36. 한국신문협회 창립 60주년 영상 축사
37. (미국 방문) CSIS 전문가 초청 만찬
38. (미국 방문) 한미 정상 공동 언론 발표
39. (미국 방문) 워싱턴 동포 간담회
40. 방미 귀국 인사
41. 제50회 산업안전보건의 날 기념식 영상 축사
42. (독일 방문) 독일 동포 간담회
43. (독일 방문) 독일 의료 지원단 접견
44. (독일 방문) 한독 정상 공동 언론 발표
45. (독일 방문) 쾨르버재단 초청 연

KeyboardInterrupt: 

In [12]:
speech_data

[{'index': 1,
  'title': '제19대 대통령 취임식',
  'type': '취임사',
  'date': '2017.05.10',
  'speaker': '문재인',
  'speech': '존경하고 사랑하는 국민 여러분!\n감사합니다. 국민 여러분의 위대한 선택에 머리 숙여 깊이 감사드립니다.\n\n저는 오늘 대한민국 제19대 대통령으로서 새로운 대한민국을 향해 첫걸음을내딛습니다. 지금 제 두 어깨는 국민 여러분으로부터 부여받은 막중한 소명감으로 무겁습니다. 지금 제 가슴은 한 번도 경험하지 못한 나라를 만들겠다는 열정으로 뜨겁습니다. 그리고 지금 제 머리는 통합과 공존의 새로운 세상을 열어 갈청사진으로 가득 차 있습니다.\n\n우리가 만들어 가려는 새로운 대한민국은 숱한 좌절과 패배에도 불구하고 우리 선대들이 일관되게 추구했던 나라입니다. 또한 많은 희생과 헌신을 감내하며 우리 젊은이들이 그토록 이루고 싶어 했던 나라입니다. 그런 대한민국을 만들기위해 저는 역사와 국민 앞에 두렵지만 겸허히 대한민국 제19대 대통령으로서 책임과 소명을 다할 것을 천명합니다.\n\n함께 선거를 치른 후보들께 감사의 말씀과 함께 심심한 위로를 전합니다. 이번 선거에서는 승자도 패자도 없습니다. 우리는 새로운 대한민국을 함께 이끌어 가야 할 동반자입니다. 이제 치열했던 경쟁의 순간을 뒤로하고 함께 손을 맞잡고 미래로 전진해야 합니다.\n\n존경하는 국민 여러분!\n지난 몇 달간 우리는 유례없는 정치적 격변기를 보냈습니다. 정치는 혼란했지만 국민은 위대했습니다. 현직 대통령의 탄핵과 구속 앞에서도 국민께서 대한민국의 앞길을 열어 주셨습니다. 우리 국민은 좌절하지 않고 오히려 전화위복의 계기로 삼아 마침내 오늘 새로운 세상을 열었습니다. 대한민국의 위대함은 국민의 위대함입니다. 그리고 이번 대통령 선거에서 우리 국민은 또 하나의 역사를 만들어 주셨습니다. 전국 각지에서 고른 지지로 새로운 대한민국을 선택해 주셨습니다. 오늘부터 저는 국민 모두의 대통령이 되겠습니다. 저